In [12]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
from numpy.linalg import norm,inv,matrix_rank
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
from sklearn.cross_decomposition import PLSRegression

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [13]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
 

877


In [14]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )


In [15]:
print(X_train.shape)
print(Y_train.shape)
n_fold = 17
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)


for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [96]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,21))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, verbose=2,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SIMPLS(),
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20]},
             scoring='neg_root_mean_squared_error', verbose=2)
Fitting 17 folds for each of 20 candidates, totalling 340 fits


[CV] END .....................................n_components=1; total time=   0.3s


[CV] END .....................................n_components=1; total time=   0.3s


[CV] END .....................................n_components=1; total time=   0.2s


[CV] END .....................................n_components=1; total time=   0.3s


...

[CV] END ....................................n_components=20; total time=   2.4s


[CV] END ....................................n_components=20; total time=   2.3s


[CV] END ....................................n_components=20; total time=   2.5s


[CV] END ....................................n_components=20; total time=   2.2s


CV elapsed time: 412.12s
best parameter:  {'n_components': 11} ; score:  -7.1476354e+00


In [95]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_list=list(range(1,21))
print("n_comp:", n_comp_list)
score_all = np.zeros((n_fold, len(n_comp_list)))

n_comp_max=20
PLS = ISIMPLS(n_components=n_comp_max)

tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(n_comp_list):
        y_pred = PLS.predict(X_train[test_index], n_comp)
        score_all[i,j] = RMSE(y_true, y_pred)
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = score_all.mean(axis=0)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {n_comp_list[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")
print(scores)

n_comp: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


fold: 01, elapsed time: 2.7s


fold: 06, elapsed time: 12.8s


fold: 11, elapsed time: 13.0s


fold: 16, elapsed time: 11.9s



best parameter: 11; score: 7.1476354e+00; total time=42.6s
[7.76705994 7.50395476 7.36823013 7.31293782 7.31652096 7.29054187
 7.2979135  7.2500481  7.23974823 7.16567089 7.14763537 7.19020176
 7.19894527 7.21953187 7.24954174 7.27576464 7.32228498 7.33506288
 7.33961495 7.38431746]


In [97]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

PLS_list = []
n_comp_max=100
PLS = ISIMPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    with open(f'./data/wrk/ISIMPLS_fold{i:02d}.pkl', 'wb') as f:
        pickle.dump(PLS, f)

    # PLS_list.append(copy.deepcopy(PLS))
    if (i%5==0):
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 16.7s


fold: 06, elapsed time: 81.9s


fold: 11, elapsed time: 77.9s


fold: 16, elapsed time: 78.5s


total time=270.4s


In [98]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

PLS_list = []
for i in range(n_fold):
    with open(f'./data/wrk/ISIMPLS_fold{i:02d}.pkl', 'rb') as f:
        PLS_list.append(pickle.load(f))


In [99]:
space  = [Integer(1, n_comp_max, name='n_components')]
@use_named_args(space)
def Comp_Model_Score(n_components):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        y_true = Y_train[test_index]
        y_pred = model[i].predict(X_train[test_index], n_components)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [103]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=n_calls*2,
                     # n_initial_points=n_calls,
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best index = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.4475
Function value obtained: 8.1527
Current minimum: 8.1527
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.6054
Function value obtained: 8.3838
Current minimum: 8.1527
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.5758
Function value obtained: 8.3889
Current minimum: 8.1527
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.5389
Function value obtained: 8.3838
Current minimum: 8.1527
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 0.7185
Function value obtained: 7.8384
Current minimum: 7.8384
Iteration No: 7 started. Evaluating function at random point.


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 0.6618
Function value obtained: 7.6128
Current minimum: 7.6128
Iteration No: 8 started. Evaluating function at random point.


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 0.5160
Function value obtained: 7.2979
Current minimum: 7.2979
Iteration No: 9 started. Evaluating function at random point.


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 0.6428
Function value obtained: 7.5730
Current minimum: 7.2979
Iteration No: 10 started. Evaluating function at random point.


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 1.6433
Function value obtained: 7.9823
Current minimum: 7.2979
----------------------------------------
Best index = 07
Best score = 7.2979
total time = 11.8s


In [36]:
tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=50, random_state=0, verbose=True)
print("-"*40)
print(f"Best index = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")

Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5842
Function value obtained: 7.1476
Current minimum: 7.1476
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.5749
Function value obtained: 7.2758
Current minimum: 7.1476
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.5773
Function value obtained: 7.2758
Current minimum: 7.1476
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.5517
Function value obtained: 7.2758
Current minimum: 7.1476
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 1.2989
Function value obtained: 7.2905
Current minimum: 7.1476
Iteration No: 49 started. Searching for the next optimal point.


/home/tingan/miniconda3/envs/pls/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [11] before, using random point [4]
  warnings.warn(


Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 1.2302
Function value obtained: 7.3129
Current minimum: 7.1476
Iteration No: 50 started. Searching for the next optimal point.


/home/tingan/miniconda3/envs/pls/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [11] before, using random point [13]
  warnings.warn(


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.3777
Function value obtained: 7.1989
Current minimum: 7.1476
----------------------------------------
Best index = 11
Best score = 7.1476
total time = 52.4s


In [ ]:
def dump_PLS(PLS, fn):
    with h5py.File(fn, 'w') as f:
        f.create_dataset('n', data=PLS.n)
        f.create_dataset('n_components', data=PLS.n_components)
        f.create_dataset('W', data=PLS.W, maxshape=PLS.W.shape, chunks=True)
        f.create_dataset('C', data=PLS.C, maxshape=PLS.C.shape, chunks=True)
        f.create_dataset('S', data=PLS.S, maxshape=PLS.S.shape, chunks=True)
        f.create_dataset('_x_mean', data=PLS._x_mean, maxshape=PLS._x_mean.shape, chunks=True)
        f.create_dataset('_y_mean', data=PLS._y_mean, maxshape=PLS._y_mean.shape, chunks=True)

def load_PLS(PLS, fn):
    with h5py.File(fn, 'r') as f:
        PLS.n = f['n'][()]
        PLS.n_components = f['n_components'][()]
        PLS.W       =f['W'][:]
        PLS.C       =f['C'][:]
        PLS.S       =f['S'][:]
        PLS._x_mean =f['_x_mean'][:]
        PLS._y_mean =f['_y_mean'][:]
    return PLS

In [ ]:
PLS_list = []

with open(f'./data/wrk/ISIMPLS_fold{n_fold-1:02d}.pkl', 'rb') as f:
    PLS_obj = pickle.load(f)

for i in range(n_fold):
    PLS_list.append(load_PLS(copy.deepcopy(PLS_obj), f'./data/wrk/ISIMPLS_fold{i:02d}.h5'))

